## Pyecharts绘图案例

### 案例数据说明

In [1]:
# 加载数据
import pandas as pd
job = pd.read_csv('data/data_analysis_job.csv')


In [2]:
# 最近一周 python相关工作数量
job.shape
job.head()
# 查看数据字段
job.columns
# company_name:公司名字，url，工作的网址，job_name : 工作名字，city：工作所在城市，salary：工资 experience：经验，company_area：公司所在行业，company_size：公司规模，description：工作描述



Index(['company_name', 'url', 'job_name', 'city', 'salary', 'experience',
       'company_area', 'company_size', 'description'],
      dtype='object')

### 哪些城市在数据分析机会多

In [3]:
# 按城市分组
city_job_list = job.groupby('city')['url'].count().sort_values(ascending = False)
city_job_top20 = city_job_list.head(20)
city_job_top20



city
北京    1608
上海    1176
广州    1033
深圳     820
杭州     395
成都     348
武汉     320
西安     248
南京     222
苏州     150
合肥     140
重庆     130
长沙     128
济南     123
郑州     114
太原     113
大连      94
青岛      94
东莞      83
无锡      81
Name: url, dtype: int64

In [4]:
# pyechart 绘制柱状图
from pyecharts import options as opts
from pyecharts.charts import Bar
c = ( Bar() #创建柱状图
    .add_xaxis(city_job_top20.index.tolist()) #添加x轴数据
    .add_yaxis('数据分析就业岗位数量', city_job_top20.values.tolist())#添加y轴数据
    .set_global_opts( #设置全局参数
        title_opts=opts.TitleOpts(title='一周内Python就业岗位数量'), #设置标题
        datazoom_opts=opts.DataZoomOpts(),#添加缩放条
    )
)



/Users/vincent/anaconda3/envs/itcast/lib/python3.6/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [5]:

c.render_notebook() # 在juypter notebook中显示


### 哪些公司在招聘Python程序员

In [6]:
# 根据公司名字对数据进行分组
company_list = job.groupby('company_name')['url'].count().sort_values(ascending = False)
company_list = company_list.head(100)
company_list


company_name
北京字节跳动科技有限公司        213
武汉益越商务信息咨询有限公司       85
北京融汇天诚投资管理有限公司       81
字节跳动                 67
腾讯科技（深圳）有限公司         43
                   ... 
北京华融泰克科技有限公司          7
广发银行股份有限公司信用卡中心       7
软通动力信息技术(集团)有限公司      7
中国平安人寿保险股份有限公司        7
墨博（湖南）文化传媒有限公司        7
Name: url, Length: 100, dtype: int64

In [7]:
# pyecharts绘制词云图
from pyecharts.charts import WordCloud
c = ( WordCloud()#创建词云图对象
    .add(series_name='哪些公司在招聘数据分析程序员', #添加标题
         data_pair=list(zip(company_list.index.tolist(),company_list.values.tolist())), 
         word_size_range=[6, 40])#指定文字大小，注意如果字体太大可能显示不全
    .set_global_opts(#设置全局参数 标题，字号
        title_opts=opts.TitleOpts(title='哪些公司在招聘数据分析程序员', 
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=23))
    )
)
c.render_notebook()


/Users/vincent/anaconda3/envs/itcast/lib/python3.6/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


### 岗位数量与平均起薪分布

In [8]:
# 数据清洗 提取起薪
import re
def get_salary_down(x):
    if ('面议' in x) or ('小时' in x) or ('以' in x):
        return 0
    elif '元' in x:
        return int(re.search(pattern="([0-9]+)-([0-9]+)元/月", string=x).group(1))
    elif '-' in x:
        return int(re.search(pattern="([0-9]+)-([0-9]+)", string=x).group(1))
    else:
        return int(x)

In [9]:
# 提取起薪数据之前，首先处理缺失值，将缺失数据去掉并提取起薪
job.dropna(subset = ['salary'],inplace = True)
job['salary_down'] = job.salary.apply(get_salary_down)
job['salary_down'].value_counts().sort_index()

0         141
1000        5
1500        8
1800        1
2000       79
         ... 
60000       3
62500       1
70000       2
80000       1
100000      1
Name: salary_down, Length: 92, dtype: int64

In [10]:
# 异常值处理：直接删除
job_salary = job[job['salary_down']>3000]
job_salary = job_salary[job_salary['salary_down']<80000]

In [11]:
# 绘制气泡图数据准备
# 将所有城市的薪资和就业岗位数合并到一起
def prepare_data(data):
    # 取出各城市有多少岗位
    temp1 = data.groupby('city')['url'].count().sort_values(ascending = False)
    # 计算出各城市岗位的平均起薪
    temp = data.groupby('city')['salary_down'].mean().sort_values(ascending = False)
    # 合并数据
    temp2 = pd.concat([temp,temp1],axis = 1)
    temp2 = temp2.reset_index()
    return temp2

In [12]:
# 处理数据
salary_data = prepare_data(job)
salary_data.columns = ['city','salary_down','job_count']
salary_data.head()

/Users/vincent/anaconda3/envs/itcast/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,city,salary_down,job_count
0,上海,10346.724490,1176
1,东莞,5163.903614,83
2,北京,13122.310945,1608
3,南京,6362.081081,222
4,南昌,5737.851852,54


In [13]:
# 气泡图展示数据
from pyecharts.charts import Scatter
from pyecharts.commons.utils import JsCode

c = (
    Scatter() #创建散点图对象
    .add_xaxis(salary_data.salary_down.astype(int))#添加x周数据（薪资）
    .add_yaxis(
        "数据分析岗位数量", #y轴数据说明
        [list(z) for z in zip(salary_data.job_count, salary_data.city)],#Y轴数据，岗位，城市
        label_opts=opts.LabelOpts(#Js代码控制气泡显示提示文字
            formatter=JsCode(
                "function(params){return params.value[2]}" #提示
            )
        ),
    )
    .set_global_opts(#全局变量
        title_opts=opts.TitleOpts(title="数据分析就业岗位数量与平均起薪"),#设置标题
        tooltip_opts=opts.TooltipOpts(#Js代码控制气泡弹窗提示文字
            formatter=JsCode(
                "function (params) {return params.value[2]+ '平均薪资：'+params.value[0]}"
            )
        ),
        visualmap_opts=opts.VisualMapOpts(#控制
            type_="size", max_=1500, min_=200, dimension=1
        ),
        xaxis_opts=opts.AxisOpts(min_=6000,name='平均起薪'),#设置X轴起始值，X轴名字
        yaxis_opts=opts.AxisOpts(min_=300,max_=1550,name='岗位数量'),#设置Y轴起始值，Y轴名字
    )
)

/Users/vincent/anaconda3/envs/itcast/lib/python3.6/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [14]:
c.render_notebook()

### 工作经验需求分析

In [15]:
# 工作经验数据清洗
job['experience'].value_counts()


1-3年       1197
1年经验        897
不限          854
3-5年        763
2年经验        555
经验3-5年      519
经验不限        483
3-4年经验      438
无需经验        387
经验1-3年      276
5-10年       206
经验5-10年     155
无经验         130
5-7年经验      103
1年以下         97
经验应届毕业生      95
10年以上        16
经验1年以下       15
8-9年经验        9
一年以下          8
经验10年以上       3
10年以上经验       2
Name: experience, dtype: int64

In [16]:
job.experience.fillna('未知',inplace = True)
def process_experience(x):
    if x in ['1-3年','2年经验','经验1-3年']:
        return '1-3年'
    elif x in ['3-5年','经验3-5年','3-4年经验']:
        return '3-5年'
    elif x in ['1年经验','1年以下','经验1年以下','一年以下','经验应届毕业生','不限','经验不限','无需经验','无经验']:
        return '一年以下/应届生/经验不限'
    elif x in ['5-10年','经验5-10年','5-7年经验','8-9年经验','10年以上经验','10年以上','经验10年以上']:
        return '5年以上'
    else:
        return x
    
job['exp'] = job.experience.apply(process_experience)
job['exp'].value_counts()

一年以下/应届生/经验不限    2966
1-3年             2028
3-5年             1720
未知                718
5年以上              494
Name: exp, dtype: int64

In [18]:
# 圆环图展示结果
from pyecharts.charts import Pie
c = (
    Pie()
    .add(
        series_name="经验要求",
        data_pair=[list(z) for z in zip(
            job['exp'].value_counts().index.tolist(),# 准备数据
            job['exp'].value_counts().values.tolist()
        )],
        radius=["50%", "70%"],#圆环图，大环小环的半径大小
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )#设置图例位置
    .set_global_opts(
        title_opts=opts.TitleOpts(title="数据分析工作经验要求"),
        legend_opts=opts.LegendOpts(pos_left="right", orient="vertical"))
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(#鼠标滑过之后弹出文字格式
            trigger="item", 
            formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
    )
)

/Users/vincent/anaconda3/envs/itcast/lib/python3.6/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [19]:
c.render_notebook()